In [49]:
import numpy as np

# generate data

In [61]:
# In real-world scenarios, learning how the data was generated is impractical. Do not rely on this function while doing research.
def generate_data(dim, num):
    x = np.random.normal(0, 10, [num, dim])
    coef = np.random.uniform(-1, 1, [dim, 1])
    pred = np.dot(x, coef)
    pred_n = (pred - np.mean(pred)) / np.sqrt(np.var(pred))
    label = np.sign(pred_n)
    mislabel_value = np.random.uniform(0, 1, num)
    mislabel = 0
    for i in range(num):
        if np.abs(pred_n[i]) < 1 and mislabel_value[i] > 0.9 + 0.1 * np.abs(pred_n[i]):
            label[i] *= -1
            mislabel += 1
    return x, label, mislabel/num

# write your model class

In [62]:
# Modifying the overall structure is acceptable but not recommended
class SVM1:
    def __init__(self, dim,C=1.0,tol=1e-4,max_iter=1000):
        """
        Adding other parameters is acceptable but not necessary.
        """
        self.dim=dim
        self.C=C
        self.tol=tol
        self.max_iter=max_iter
        self.alpha=None
        self.bias=None

    def _compute_weight(self,alpha,X,y):
        return np.sum(alpha[i]*y[i]*x[i] for i in range(len(alpha)))
    
    def _compute_bias(self,alpha,X,y,w):
        #对应支持向量的位置
        support_indice=np.where((alpha>1e-5))[0]
        return np.mean(y[support_indice]-np.dot(X[support_indice],w))

    
    def _duiou_function(self,alpha):
        #目标函数
        return 0.5* np.sum(alpha **2)-np.sum(alpha)

    def _constraint(self,alpha,y):
        #约束函数
        return np.sum(alpha*self.y)

    def fit(self, X, y):
        """
        Fit the coefficients via your method1
        """
        self.X=X
        self.y=y
        tol=self.tol
        C=self.C
        num=X.shape[0]

        alpha=np.zeros(num)
        bias=0
        iter_times=0

        while iter_times< self.max_iter:
            alpha_prev=np.copy(alpha)

            for i in range(num):
                #E_i表示f(x)
                E_i=np.dot(alpha*y,(np.dot(X,X[i].T)))+bias-y[i]
                if(y[i]*E_i<-self.tol and alpha[i]<self.C) or (y[i]*E_i>self.tol and alpha[i]>0):
                    j=np.random.choice(num)
                    while j==i:
                        j=np.random.choice(num)
                    
                    E_j=np.dot(alpha*y,(np.dot(X,X[j].T)))+bias-y[j]
                    alpha_i_prev,alpha_j_prev=alpha[i],alpha[j]

                    if y[i]!=y[j]:
                        L=max(0,alpha[j]-alpha[i])
                        H=min(C,C+alpha[j]-alpha[i])
                    else:
                        L=max(0,alpha[i]+alpha[j]-C)
                        H=min(C,alpha[i]+alpha[j])
                
                    if L==H:
                        continue

                    eta= 2*np.dot(X[i],X[j].T)-np.dot(X[i],X[i].T)-np.dot(X[j],X[j].T)
                    if eta>=0:
                        continue

                    alpha[j]=alpha[j]-(y[j]*(E_i-E_j))/eta
                    alpha[j]=max(L,min(H,alpha[j]))

                    if abs(alpha_j_prev-alpha[j])<1e-5:
                        continue

                    alpha[i]=alpha[i]+y[i]*y[j]*(alpha_j_prev-alpha[j])

                    b1=bias-E_i + y[i]*(alpha_i_prev-alpha[i])*np.dot(X[i],X[i].T)-y[j]*(alpha[j]-alpha_j_prev)*np.dot(X[i],X[j].T)
                    b2=bias-E_j + y[i]*(alpha_i_prev-alpha[i])*np.dot(X[i],X[j].T)-y[j]*(alpha[j]-alpha_j_prev)*np.dot(X[j],X[i].T)

                    if 0<alpha[i]<C:
                        bias=b1
                    if 0<alpha[i]<C:
                        bias=b2

                    else:
                        bias=(b1+b2)/2
                
            iter_times+=1

            if np.linalg.norm(alpha-alpha_prev)<tol:
                break

            self.alpha=alpha
            w=self._compute_weight(alpha,X,y)
            self.bias=self._compute_bias(alpha,X,y,w)

            

        
    def predict(self, X):
        """
        Generate prediction probabilities on a new
        collection of data points by your model.
        """
        return np.sign(np.dot(X,self._compute_weight(self.alpha,self.X,self.y))+self.bias)

In [67]:

np.random.seed(42)  # For reproducibility
x, y, mislabel_rate = generate_data(25, 10000)

# Split data (80% training, 20% testing)
data_nums = y.shape[0]
y = y.reshape(data_nums)
split_idx = int(0.8 * len(x))
X_train, y_train = x[:split_idx], y[:split_idx]
X_test, y_test = x[split_idx:], y[split_idx:]


In [64]:
import time
model1 = SVM1(dim=20, C=1.0, tol=1e-3, max_iter=100)
svmtime1_a=time.time()
model1.fit(X_train, y_train)
pred1 = model1.predict(X_test)
svmtime1_b=time.time()
print("SVM1 accuracy on X_train:", np.mean(model1.predict(X_train) == y_train))
print("SVM1 accuracy on X_test:", np.mean(model1.predict(X_test) == y_test))
print('SVM1 time for fitting and predict:%fs' % (svmtime1_b - svmtime1_a)) 

C:\Users\CindyWu\AppData\Local\Temp\ipykernel_29524\1259433648.py:15: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  return np.sum(alpha[i]*y[i]*x[i] for i in range(len(alpha)))


SVM1 accuracy on X_train: 0.882125
SVM1 accuracy on X_test: 0.867
SVM1 time for fitting and predict:45.823442s


In [69]:

class SVM2:
    def __init__(self, dim, learning_rate=0.00001, epochs=1000):
        """
        Adding other parameters is acceptable but not necessary.
        """
        self.dim = dim
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.weights = np.zeros(dim)

    def fit(self, X, y):
        """
        Fit the coefficients via your method2
        """
        for epoch in range(self.epochs):
            # Calculate predictions and hinge loss
            predictions = np.dot(X, self.weights)
            hinge_loss = np.maximum(0, 1 - y * predictions)

            # Update weights with gradient descent
            gradient = -np.dot(X.T, y * (hinge_loss > 0))
            self.weights -= self.learning_rate * gradient
        
    def predict(self, X):
        """
        A same predict function with SVM1 is acceptable.
        """
        predictions = np.dot(X, self.weights)
        return np.sign(predictions)#.flatten()

# construct and train your models

In [7]:
# generate data
# X_data, y_data, mislabel = generate_data() 

# split data


# constrcut model and train (remember to record your time consumption)
# model1 = SVM1() 
# model1.fit()


In [110]:
np.random.seed(42) 
x, y, mislabel_rate = generate_data(25, 10000)

data_nums = y.shape[0]
y = y.reshape(data_nums)
split_idx = int(0.8 * len(x))
X_train, y_train = x[:split_idx], y[:split_idx]
X_test, y_test = x[split_idx:], y[split_idx:]

# predict and compare your results

In [8]:
# make prediction
# pred = model1.predict()

# compare with generated label

# compare each method

# (Optional) compare with sklearn

In [122]:
import time
model1 = SVM1(dim=25, C=1.0, tol=1e-4, max_iter=50)
svmtime1_a=time.time()
model1.fit(X_train, y_train)
svmtime1_b=time.time()
pred1 = model1.predict(X_test)
print("SVM1 accuracy on X_train:", np.mean(model1.predict(X_train) == y_train))
print("SVM1 accuracy on X_test:", np.mean(model1.predict(X_test) == y_test))
print('SVM1 time for fitting:%fs' % (svmtime1_b - svmtime1_a)) 

C:\Users\CindyWu\AppData\Local\Temp\ipykernel_29524\1259433648.py:15: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  return np.sum(alpha[i]*y[i]*x[i] for i in range(len(alpha)))


SVM1 accuracy on X_train: 0.90775
SVM1 accuracy on X_test: 0.908
SVM1 time for fitting:31.383779s


In [112]:
mislabel_rate

0.0353

In [117]:
model2 = SVM2(dim=25, learning_rate=1e-5, epochs=2000)
svmtime2_a=time.time()
model2.fit(X_train, y_train)
svmtime2_b=time.time()
pred2 = model2.predict(X_test)
print("SVM2 accuracy on X_train:", np.mean(model2.predict(X_train) == y_train))
print("SVM2 accuracy on X_test:", np.mean(model2.predict(X_test) == y_test))
print('SVM2 time for fitting: %fs' % (svmtime2_b - svmtime2_a)) 

SVM2 accuracy on X_train: 0.959125
SVM2 accuracy on X_test: 0.9655
SVM2 time for fitting and predict: 0.150336s


In [115]:
from sklearn import svm
standard = svm.SVC()  
svmtime_sklearn_a=time.time()
standard.fit(X_train,y_train)
svmtime_sklearn_b = time.time()
print("sklearn accuracy on X_train:",standard.score(X_train, y_train))
print("sklearn accuracy on X_test:",standard.score(X_test, y_test))   
print('sklearn time %fs' % (svmtime_sklearn_b - svmtime_sklearn_a)) 

sklearn accuracy on X_train: 0.971
sklearn accuracy on X_test: 0.9465
sklearn time 0.903238s
